# Задание по программированию: Анализ текстов

## Вы научитесь:
- находить оптимальные параметры для метода опорных векторов
- работать с текстовыми данными

## Введение
Метод опорных векторов (Support Vector Machine, SVM) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на **разреженных данных**. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а <a href='https://ru.wikipedia.org/wiki/TF-IDF'>TF-IDF</a>. Это показатель, который равен произведению двух чисел: `TF (term frequency)` и `IDF (inverse document frequency)`. Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, **TF-IDF** будет иметь высокое значение для тех слов, которые **много раз встречаются в данном документе, и редко встречаются в остальных**.

## Данные
Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

## Реализация в Scikit-Learn
Для начала вам потребуется загрузить данные. В этом задании мы воспользуемся одним из датасетов, доступных в scikit-learn'е — **20 newsgroups**. Для этого нужно воспользоваться модулем `datasets`:

После выполнения этого кода массив с текстами будет находиться в поле `newsgroups.data`, номер класса — в поле `newsgroups.target`.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление `TF-IDF`. В Scikit-Learn это реализовано в классе `sklearn.feature_extraction.text.TfidfVectorizer`. Преобразование обучающей выборки нужно делать с помощью функции `fit_transform`, тестовой — с помощью `transform`.

Реализация SVM-классификатора находится в классе `sklearn.svm.SVC`. Веса каждого признака у обученного классификатора хранятся в поле `coef_`. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом `get_feature_names()` у `TfidfVectorizer`:

Подбор параметров удобно делать с помощью класса sklearn.grid_search.GridSearchCV (При использовании библиотеки scikit-learn версии 18.0.1 sklearn.model_selection.GridSearchCV). Пример использования:

При использовании библиотеки scikit-learn версии 18.0.1 и выше KFold задаётся немного по-другому:

Первым аргументом в GridSearchCV передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (dict), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

## Инструкция по выполнению
1) Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [98]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

In [117]:
# функция для записи ответов
def write_answer(name, answer):
    with open('data/' + name + '.txt', 'w') as file:
        file.write(str(answer))

In [13]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

In [14]:
 newsgroups.data[0]

'From: 9051467f@levels.unisa.edu.au (The Desert Brat)\nSubject: Re: Keith Schneider - Stealth Poster?\nOrganization: Cured, discharged\nLines: 24\n\nIn article <1pa0f4INNpit@gap.caltech.edu>, keith@cco.caltech.edu (Keith Allan Schneider) writes:\n\n> But really, are you threatened by the motto, or by the people that use it?\n\nEvery time somone writes something and says it is merely describing the norm,\nit is infact re-inforcing that norm upon those programmed not to think for\nthemselves. The motto is dangerous in itself, it tells the world that every\n*true* American is god-fearing, and puts down those who do not fear gods. It\ndoesn\'t need anyone to make it dangerous, it does a good job itself by just\nexisting on your currency.\n\n> keith\n\nThe Desert Brat\n-- \nJohn J McVey, Elc&Eltnc Eng, Whyalla, Uni S Australia,    ________\n9051467f@levels.unisa.edu.au      T.S.A.K.C.            \\/Darwin o\\\nFor replies, mail to whjjm@wh.whyalla.unisa.edu.au      /\\________/\nDisclaimer:

In [15]:
newsgroups.target[:4]

array([0, 0, 1, 1], dtype=int64)

In [35]:
data = newsgroups.data
y = newsgroups.target

2) Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по **всем** данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [36]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

In [31]:
X

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

3) Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [38]:
%%time
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

Wall time: 8min 22s


In [121]:
gs.best_params_

{'C': 1.0}

4) Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [41]:
# это просто лучший определенный на предыдущем шаге классификаор
gs.best_estimator_

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

5) Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

**1.** "вытащим" массив коэффициентов и преобразуем его из формата sparse в обычный массив

In [96]:
gs.best_estimator_.coef_.toarray()

array([[ 0.29258057, -0.12314757,  0.        , ...,  0.01972862,
         0.05831336, -0.00297347]])

**2.** отсортируем **минус** модули коэффициэнтов ("минус" - для сортировки по убыванию), возьмем их индексы (функция `np.argsort`) и выделим первые 10 индексов для перевода их в слова

In [152]:
word10_index = np.argsort(-np.abs(gs.best_estimator_.coef_.toarray()))[0, :10]
word10_index

array([24019, 12871,  5088,  5093, 17802, 23673, 21850,  5776, 15606, 22936], dtype=int64)

для примера попробуем применения преобразования индекса слова в слово

In [143]:
feature_mapping = vectorizer.get_feature_names()
print(feature_mapping[4890])

article


**3.** преобразуеем индексы в слова и отсортируем

In [154]:
word10 = sorted([feature_mapping[ind] for ind in word10_index])
word10

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'religion',
 'sci',
 'sky',
 'space']

In [145]:
write_answer('svm-texts_answer1', ' '.join(word10))